In [1]:
!conda install -y gdown

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.1.2
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    filelock-3.13.3            |     pyhd8ed1ab_0          15 KB  conda-forge
    gdown-5.1.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.2.1              |       hd590300_1         2.7 MB  conda-forge
    ------------------------------------------------------------
                                    

In [2]:
!gdown --id 1yqkIVhUwInBzefGWfcAskmIrvKWtMCRZ

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1yqkIVhUwInBzefGWfcAskmIrvKWtMCRZ
From (redirected): https://drive.google.com/uc?id=1yqkIVhUwInBzefGWfcAskmIrvKWtMCRZ&confirm=t&uuid=573a287a-8a09-4ac7-b654-e592d862e83a
To: /kaggle/working/dataset.csv
100%|███████████████████████████████████████| 8.25G/8.25G [01:08<00:00, 121MB/s]


In [3]:
import pandas as pd
import math
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [4]:
data = pd.read_csv('./dataset.csv')


In [5]:
data.columns=["user_ip","domain","timestamp","attack","request","len","subdomains_count","w_count","w_max","entropy","w_max_ratio","w_count_ratio","digits_ratio","uppercase_ratio","time_avg","time_stdev","size_avg","size_stdev","throughput","unique","entropy_avg","entropy_stdev"]
data.head()

,user_ip,domain,timestamp,attack,request,len,subdomains_count,w_count,w_max,entropy,...,digits_ratio,uppercase_ratio,time_avg,time_stdev,size_avg,size_stdev,throughput,unique,entropy_avg,entropy_stdev
0,186.169.253.58,surbl.org,1624438272607,False,f.surbl.org,1,1,0,0,-0.000000,...,0.000000,0.0,0.222222,0.440959,1.0,0.000000,3333.333333,0.555556,0.000000,0.000000
1,186.169.253.58,spamhaus.org,1624438273058,False,118.141.11.106.sbl.spamhaus.org,18,5,0,0,2.633731,...,0.611111,0.0,55.555556,165.542375,17.2,0.421637,343.313373,0.000000,3.048277,0.177285
2,186.169.253.58,spamhaus.org,1624438273058,False,118.141.11.106.zen.spamhaus.org,18,5,1,3,2.633731,...,0.611111,0.0,0.333333,0.500000,17.2,0.421637,43000.000000,0.000000,2.983547,0.199622
3,186.169.253.58,spamhaus.org,1624438273059,False,128.141.11.106.sbl.spamhaus.org,18,5,0,0,2.863826,...,0.611111,0.0,0.333333,0.500000,17.3,0.483046,43250.000000,0.000000,2.959741,0.198131
4,186.169.253.58,spamhaus.org,1624438273059,False,128.141.11.106.zen.spamhaus.org,18,5,1,3,2.863826,...,0.611111,0.0,0.333333,0.500000,17.4,0.516398,43500.000000,0.000000,2.935936,0.193400


In [6]:
data["attack"]=data["attack"].astype(int)
n=len(data[data['attack'] == 1])
label1 = data[data['attack'] == 1]
label0 = data[data['attack'] == 0].sample(n, random_state=42)
data = pd.concat([label1, label0])
data.head()

,user_ip,domain,timestamp,attack,request,len,subdomains_count,w_count,w_max,entropy,...,digits_ratio,uppercase_ratio,time_avg,time_stdev,size_avg,size_stdev,throughput,unique,entropy_avg,entropy_stdev
3942,186.169.146.147,e5.sk,1624438294225,1,sebubx76xk4erpp3rwehoo3ubmbqeaqbaeaq.a.e.e5.sk,40,3,3,3,3.975071,...,0.125,0.0,2197.222222,2875.261022,48.2,53.370404,24.372977,0.0,3.691242,0.910175
4297,186.169.146.147,e5.sk,1624438295586,1,4az3kiecotwu3okbtvfm7pdpcabqeaqbaeaq.a.e.e5.sk,40,3,5,3,4.146439,...,0.100,0.0,2348.444444,2779.448601,48.2,53.370404,22.803615,0.0,3.685581,0.906808
4590,186.169.146.147,e5.sk,1624438296656,1,x3i2wbqsiucuviqyfaaoxz3lzybqeaqbaeaq.a.e.e5.sk,40,3,1,3,3.987326,...,0.075,0.0,2460.111111,2695.151964,51.8,51.228898,23.394454,0.0,3.884313,0.687639
6096,186.169.127.58,e5.sk,1624438302237,1,ez2vzwchw3ce5m6wz6cw3nnc2ibqeaqbaeaq.a.e.e5.sk,40,3,1,3,3.893943,...,0.175,0.0,1799.222222,1935.781934,44.0,27.712813,27.170557,0.0,3.835620,0.663023
6187,186.169.146.147,e5.sk,1624438302672,1,htm7xrligq2enc4lsjhkzdnd6mbqeaqbaeaq.a.e.e5.sk,40,3,3,3,4.371928,...,0.100,0.0,3105.444444,2782.422466,51.8,51.228898,18.533095,0.0,3.905225,0.700116


In [7]:
print(len(data[data['attack'] == 1]))
print(len(data[data['attack'] == 0]))
X = data[["len","subdomains_count","w_count","w_max","entropy","w_max_ratio","w_count_ratio","digits_ratio","uppercase_ratio","time_avg","time_stdev","size_avg","size_stdev","throughput","unique","entropy_avg","entropy_stdev"]]
Y = data["attack"]

174779
174779


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=29)

X_train, Y_train = shuffle(X_train, Y_train)
X_test, Y_test = shuffle(X_test, Y_test)

In [9]:
from sklearn.tree import DecisionTreeClassifier
model_dtc = DecisionTreeClassifier()
model_dtc.fit(X_train, Y_train)

DecisionTreeClassifier()

In [10]:
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(X_train, Y_train)

GaussianNB()

In [11]:
y_preds_dtc = model_dtc.predict(X_test)
print("Detection Accuracy: ",accuracy_score(Y_test, y_preds_dtc)*100,"%")
print("Classification Report:")
print(classification_report(Y_test, y_preds_dtc))

Detection Accuracy:  99.92705115001716 %
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     34941
           1       1.00      1.00      1.00     34971

    accuracy                           1.00     69912
   macro avg       1.00      1.00      1.00     69912
weighted avg       1.00      1.00      1.00     69912



In [12]:
y_preds_nb = model_nb.predict(X_test)
print("Detection Accuracy: ",accuracy_score(Y_test, y_preds_nb)*100,"%")
print("Classification Report:")
print(classification_report(Y_test, y_preds_nb))

Detection Accuracy:  91.37773200595034 %
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92     34941
           1       0.96      0.86      0.91     34971

    accuracy                           0.91     69912
   macro avg       0.92      0.91      0.91     69912
weighted avg       0.92      0.91      0.91     69912

